In [ ]:
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
vent = pd.read_csv('../vent_anon.csv')
vent.head()

In [ ]:
copd = pd.read_csv('../copd_no_phi.csv')

In [ ]:
copd10 = ['J41.0', 'J41.1', 'J41.8', 'J42', 'J43.0', 'J43.1', 'J43.2', 'J43.8', 'J43.9', 'J44.0', 'J44.1', 'J44.9', 'J98.2', 'J98.3']
copd9 = ['466.0', '491.20', '491.21', '491.22', '492.8', '493.20', '493.21', '493.22', '494.0', '494.1', '495.9', '496']

In [ ]:
icd9 = []
for i in copd9:
    icd9.append(float(i))

In [ ]:
copd_patient = np.unique(copd.loc[copd['ICD-9 Diagnosis'].isin(icd9), 'empi_anon'].values)

len(copd_patient)

In [ ]:
# def pull_data(base_path, data_type):
    
#     years = [os.path.join(base_path, x) for x in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, x))]
#     proc_files = []
#     for path in years:
#         tmp = os.listdir(path)
#         for fi in tmp:
#             if data_type.lower() in fi.lower() and ('2019' in fi.lower() or '2020' in fi.lower()):
#                 proc_files.append(os.path.join(path, fi))
#     print(proc_files)
#     data = pd.DataFrame()
    

#     for file in tqdm(proc_files):
#         tmp = pd.read_csv(file)
#         data = pd.concat([data, tmp], axis=0, ignore_index=True)
    
#     data = data.drop_duplicates() #removes date overlaps
#     return data

In [ ]:
# diagnosis = pull_data('/data/EM_HOSP_DATA/', 'DIAGNOSIS')

In [ ]:
# diagnosis.loc[diagnosis[(diagnosis['DX_CODE_ICD9'].isin(copd9)) | (diagnosis['DX_CODE_ICD10'].isin(copd10))].index, 'outcome'] = 1
# diagnosis['outcome'].fillna(0, inplace=True)

In [ ]:
# copd_empis = diagnosis[diagnosis['outcome'] == 1]['empi_anon'].unique()

In [ ]:
# len(copd_empis)

In [ ]:
new_vent_types = ['Other: NC', 'Other: hfnc', 'Other: NIV refused',
'Other: pt on ra...in no respiratory distress..bipap not in croom', 'Other: Room air',
 'Other: none', 'Other: patient is confused and refues to wear cap', 'Other: patient is confused and refuse to wear cap'
 'Other: PT refused to wear', 'Other: pt refused', 'Other: hi flow nc',
 'Other: Patient extubated per CCM placed on 3l/m nasal cannula','Other: extubated',
 'Other: pt home unit, pt not wearing']

In [ ]:
images_to_exclude = vent.loc[~vent['VENT_TYPE'].isin(new_vent_types)]

In [ ]:
# images_to_exclude = pd.DataFrame(columns=['empi_anon', 'vent_type', 'study_date_anon'])
# for i, r in vent.iterrows():
#     vent_status = r['VENT_TYPE']
#     ID = r['empi_anon']
    
#     if not (vent_status in new_vent_types):
#         images_to_exclude = images_to_exclude.append(pd.DataFrame({'empi_anon':ID, 'vent_type':vent_status, 'study_date_anon':r['recorded_time_anon']}, index=[0]), ignore_index=True)
#     else:
#         continue

In [ ]:
cxr_meta = pd.read_csv('/data/Emory_CXR_Project/EM_CXR_META_ANON_FINAL.csv')

In [ ]:
for ID in np.unique(images_to_exclude['empi_anon'].values):
    invalid_date = images_to_exclude.loc[images_to_exclude['empi_anon']==ID, 'recorded_time_anon'].values
    
    temp = cxr_meta.loc[cxr_meta['empi_anon']==ID]
    idx = temp[temp['StudyDate_anon'].isin(invalid_date)].index
    
    if (len(idx) > 0):
        cxr_meta = cxr_meta.drop(idx)
    else:
        continue

In [ ]:
cxr_meta.to_csv('CXR_META_FILTERED.csv')

In [ ]:
cxr_meta = pd.read_csv('CXR_META_FILTERED.csv')

In [ ]:
gender_array = []
copd_array = []
races_array = []
age_array = []
for i in np.unique(cxr_meta['empi_anon'].values):
    row = cxr_meta.loc[cxr_meta['empi_anon']==i]
    
    if (np.unique(row['Gender'])=='Male'):
        gender_array.append(1)
    else:
        gender_array.append(0)
        
    if (np.unique(row['Race'])=='WHITE'):
        races_array.append(0)
    elif (np.unique(row['Race'])=='BLACK/AFRICAN AMERICAN'):
        races_array.append(1)
    elif (np.unique(row['Race'])=='HISPANIC/LATINO'):
        races_array.append(2)
    elif (np.unique(row['Race'])=='OTHER' or np.unique(row['Race'])=='NATIVE'):
        races_array.append(3)
    elif (np.unique(row['Race'])=='ASIAN'):
        races_array.append(4)
    else:
        races_array.append(3)
    
    if (np.unique(row['age'])=='0-20'):
        age_array.append(0)
    elif (np.unique(row['age'])=='20-40'):
        age_array.append(1)
    elif (np.unique(row['age'])=='40-60'):
        age_array.append(2)
    elif (np.unique(row['age'])=='60-80'):
        age_array.append(3)
    else:
        age_array.append(4)
        
    if (np.unique(row['empi_anon']) in copd_patient):
        copd_array.append(1)
    else:
        copd_array.append(0)

In [ ]:
len(np.where(np.array(copd_array)==1)[0])/len(np.where(np.array(copd_array)==0)[0])

In [ ]:
from tableone import TableOne

df = pd.DataFrame({'subject_id':np.unique(cxr_meta['empi_anon'].values), 'gender': gender_array, 
                              'age':age_array, 'race': races_array,
                              'COPD':copd_array
                             })

columns = ['gender', 'race', 'age']
categorical = ['gender', 'race', 'age']
groupby = 'COPD'
mytable = TableOne(df, columns=columns, categorical=categorical,
                   groupby=groupby, pval=True)

mytable

In [ ]:
from sklearn.model_selection import train_test_split

subject_ids = np.unique(np.array(cxr_meta['empi_anon']))
train, test = train_test_split(subject_ids, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
def get_age_interval(age):
    if (age == '0-20' or age == '20-40'):
        return 0
    elif (age == '40-60'):
        return 1
    elif (age == '60-80'):
        return 2
    elif (age == '80-'):
        return 3
    else:
        return 3

def get_gender(gender):
    if (gender=='Female'):
        return 0
    elif (gender == 'Male'):
        return 1
    else:
        return 1
    
def get_race(race):
    if (race == 'WHITE'):
        return 0
    elif (race == 'BLACK/AFRICAN AMERICAN'):
        return 1
    elif (race == 'HISPANIC/LATINO'):
        return 2
    elif (race == 'OTHER' or race == 'NATIVE'):
        return 3
    elif (race == 'ASIAN'):
        return 4
    else:
        return 3

In [ ]:
from PIL import Image, ImageOps
import io

imgs = []
gender = []
age = []
race = []
copd = []
subject_id = []

for i in train:  
    r = cxr_meta.loc[cxr_meta['empi_anon']==i]
    
    try:
        for j in r['hiti_path'].values:
            img = Image.open(j)

            img = img.resize((256,256), Image.ANTIALIAS)

            img = ImageOps.equalize(img)
            imgByteArr = io.BytesIO()
            img.save(imgByteArr, format='JPEG')
            imgByteArr = imgByteArr.getvalue()
            imgs.append(imgByteArr)

            gender.append(get_gender(np.unique(r['Gender'].values)))
            age.append(get_age_interval(np.unique(r['age'].values)))
            race.append(get_race(np.unique(r['Race'].values)))
            subject_id.append(np.unique(r['empi_anon']))

            if (np.unique(r['empi_anon']) in copd_patient):
                copd.append(1)
            else:
                copd.append(0)

    except:
        continue

In [ ]:
import tensorflow as tf

record_file_test = 'tfrecords/copd_emory_train.tfrecords'

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in range(len(imgs)):
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(imgs[i])  

        example.features.feature['COPD'].int64_list.value.append(copd[i])
                
        example.features.feature['race'].int64_list.value.append(race[i])
        
        example.features.feature['age'].int64_list.value.append(age[i])
        
        example.features.feature['gender'].int64_list.value.append(gender[i])
        
        example.features.feature['subject_id'].int64_list.value.append(subject_id[i])
        
        # append label to the example
        
        writer.write(example.SerializeToString())

In [ ]:
df

In [ ]:
from PIL import Image, ImageOps
import io

imgs = []
gender = []
age = []
race = []
copd = []
path = []
subject_id = []


icd = [466., 491.2, 491.21, 491.22, 492.8, 493.2, 493.21, 493.22, 494., 494.1, 495.9, 496]

for i in val:  
    r = cxr_meta.loc[cxr_meta['empi_anon']==i]
    
    try:
        for j in r['hiti_path'].values:
            img = Image.open(j)

            img = img.resize((256,256), Image.ANTIALIAS)

            img = ImageOps.equalize(img)
            imgByteArr = io.BytesIO()
            img.save(imgByteArr, format='JPEG')
            imgByteArr = imgByteArr.getvalue()
            imgs.append(imgByteArr)

            gender.append(get_gender(np.unique(r['Gender'].values)))
            age.append(get_age_interval(np.unique(r['age'].values)))
            race.append(get_race(np.unique(r['Race'].values)))
            subject_id.append(np.unique(r['empi_anon']))
            path.append(j)

            if (np.unique(r['empi_anon']) in copd_patient):
                copd.append(1)
            else:
                copd.append(0)

    except:
        continue

In [ ]:
import tensorflow as tf

record_file_test = 'tfrecords/copd_emory_val.tfrecords'

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in range(len(imgs)):
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(imgs[i])  

        example.features.feature['COPD'].int64_list.value.append(copd[i])
                
        example.features.feature['race'].int64_list.value.append(race[i])
        
        example.features.feature['age'].int64_list.value.append(age[i])
        
        example.features.feature['gender'].int64_list.value.append(gender[i])
        
        example.features.feature['subject_id'].int64_list.value.append(subject_id[i])
        
        example.features.feature['hiti_path'].bytes_list.value.append(path[i].encode('utf-8'))
        
        # append label to the example
        
        writer.write(example.SerializeToString())

In [ ]:
from PIL import Image, ImageOps
import io
imgs = []
gender = []
age = []
race = []
copd = []
path = []
subject_id = []

icd = [466., 491.2, 491.21, 491.22, 492.8, 493.2, 493.21, 493.22, 494., 494.1, 495.9, 496]

for i in test:  
    r = cxr_meta.loc[cxr_meta['empi_anon']==i]
    
    try:
        for j in r['hiti_path'].values:
            img = Image.open(j)

            img = img.resize((256,256), Image.ANTIALIAS)

            img = ImageOps.equalize(img)
            imgByteArr = io.BytesIO()
            img.save(imgByteArr, format='JPEG')
            imgByteArr = imgByteArr.getvalue()
            imgs.append(imgByteArr)

            gender.append(get_gender(np.unique(r['Gender'].values)))
            age.append(get_age_interval(np.unique(r['age'].values)))
            race.append(get_race(np.unique(r['Race'].values)))
            subject_id.append(np.unique(r['empi_anon'].values))
            path.append(j)

            if (np.unique(r['empi_anon']) in copd_patient):
                copd.append(1)
            else:
                copd.append(0)

    except:
        continue

In [ ]:
import tensorflow as tf

record_file_test = 'tfrecords/copd_emory_test2.tfrecords'

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in range(len(imgs)):
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(imgs[i])  

        example.features.feature['COPD'].int64_list.value.append(copd[i])
                
        example.features.feature['race'].int64_list.value.append(race[i])
        
        example.features.feature['age'].int64_list.value.append(age[i])
        
        example.features.feature['gender'].int64_list.value.append(gender[i])
        
        example.features.feature['subject_id'].int64_list.value.append(subject_id[i])
        
        example.features.feature['hiti_path'].bytes_list.value.append(path[i].encode('utf-8'))
        
        # append label to the example
        
        writer.write(example.SerializeToString())

In [ ]:
filename = ['copd_emory_test.tfrecords', 'copd_emory_train.tfrecords', 'copd_emory_val.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filename)
copd = []
age = []
gender = []
race = []
subject_id = []

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 
    
    copd.append(example.features.feature['COPD'].int64_list.value[0])
    race.append(example.features.feature['race'].int64_list.value[0])
    
    try:
        age.append(example.features.feature['age'].int64_list.value[0])
        gender.append(example.features.feature['gender'].int64_list.value[0])
        subject_id.append(example.features.feature['subject_id'].int64_list.value[0])
    except:
        age.append(-1)
        gender.append(-1)
        subject_id.append(-1)

In [ ]:
len(np.where(np.array(copd)==1)[0])

In [ ]:
imgs = []
gender = []
age = []
race = []
copd = []
subject_id = []

split = 'train'

filename = ['tfrecords/copd_emory_{i}.tfrecords'.format(i=split), 'tfrecords/copd_mimic_{i}.tfrecords'.format(i=split)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    imgs.append(example.features.feature['jpg_bytes'].bytes_list.value[0])
    gender.append(example.features.feature['gender'].int64_list.value[0])
    race.append(example.features.feature['race'].int64_list.value[0])
    age.append(example.features.feature['age'].int64_list.value[0])
    subject_id.append(example.features.feature['subject_id'].int64_list.value[0])
    copd.append(example.features.feature['COPD'].int64_list.value[0])

idx = np.arange(len(subject_id))
np.random.shuffle(idx)

record_file_test = 'tfrecords/copd_merged_{i}.tfrecords'.format(i=split)

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in idx:
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(imgs[i])  

        example.features.feature['COPD'].int64_list.value.append(copd[i])
                
        example.features.feature['race'].int64_list.value.append(race[i])
        
        example.features.feature['age'].int64_list.value.append(age[i])
        
        example.features.feature['gender'].int64_list.value.append(gender[i])
        
        # append label to the example
        
        writer.write(example.SerializeToString())

In [ ]:
imgs = []
gender = []
age = []
race = []
copd = []
subject_id = []

split = 'test'

filename = ['tfrecords/copd_emory_{i}.tfrecords'.format(i=split), 'tfrecords/copd_mimic_{i}.tfrecords'.format(i=split)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    imgs.append(example.features.feature['jpg_bytes'].bytes_list.value[0])
    gender.append(example.features.feature['gender'].int64_list.value[0])
    race.append(example.features.feature['race'].int64_list.value[0])
    age.append(example.features.feature['age'].int64_list.value[0])
    subject_id.append(example.features.feature['subject_id'].int64_list.value[0])
    copd.append(example.features.feature['COPD'].int64_list.value[0])

idx = np.arange(len(subject_id))
np.random.shuffle(idx)

record_file_test = 'tfrecords/copd_merged_{i}.tfrecords'.format(i=split)

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in idx:
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(imgs[i])  

        example.features.feature['COPD'].int64_list.value.append(copd[i])
                
        example.features.feature['race'].int64_list.value.append(race[i])
        
        example.features.feature['age'].int64_list.value.append(age[i])
        
        example.features.feature['gender'].int64_list.value.append(gender[i])
        
        # append label to the example
        
        writer.write(example.SerializeToString())

In [ ]:
imgs = []
gender = []
age = []
race = []
copd = []
subject_id = []

split = 'val'

filename = ['tfrecords/copd_emory_{i}.tfrecords'.format(i=split), 'tfrecords/copd_mimic_{i}.tfrecords'.format(i=split)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    imgs.append(example.features.feature['jpg_bytes'].bytes_list.value[0])
    gender.append(example.features.feature['gender'].int64_list.value[0])
    race.append(example.features.feature['race'].int64_list.value[0])
    age.append(example.features.feature['age'].int64_list.value[0])
    subject_id.append(example.features.feature['subject_id'].int64_list.value[0])
    copd.append(example.features.feature['COPD'].int64_list.value[0])

idx = np.arange(len(subject_id))
np.random.shuffle(idx)

record_file_test = 'tfrecords/copd_merged_{i}.tfrecords'.format(i=split)

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in idx:
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(imgs[i])  

        example.features.feature['COPD'].int64_list.value.append(copd[i])
                
        example.features.feature['race'].int64_list.value.append(race[i])
        
        example.features.feature['age'].int64_list.value.append(age[i])
        
        example.features.feature['gender'].int64_list.value.append(gender[i])
        
        # append label to the example
        
        writer.write(example.SerializeToString())

In [ ]:
Races = ['WHITE', 'BLACK/AFRICAN AMERICAN', 'HISPANIC/LATINO', 'OTHER', 'ASIAN']
Age = ['20-40', '40-60', '60-80', '80+']
Gender = ['Female', 'Male']

In [ ]:
import tensorflow as tf

filename = ['tfrecords/copd_emory_train.tfrecords', 'tfrecords/copd_emory_test.tfrecords', 'tfrecords/copd_emory_val.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filename)
copds = []
ages = []
genders = []
races = []
subject_ids = []
copds_all = [0, 0]

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 
    
    subject_id = example.features.feature['subject_id'].int64_list.value[0]
    copds_all[example.features.feature['COPD'].int64_list.value[0]] += 1
    if (subject_id in subject_ids):
        continue
    
    copds.append(example.features.feature['COPD'].int64_list.value[0])
    races.append(Races[example.features.feature['race'].int64_list.value[0]])
    ages.append(Age[example.features.feature['age'].int64_list.value[0]])
    genders.append(Gender[example.features.feature['gender'].int64_list.value[0]])
    subject_ids.append(subject_id)

In [ ]:
from tableone import TableOne

df = pd.DataFrame({'subject_id':subject_ids, 'gender': genders, 
                              'age':ages, 'race': races,
                              'COPD':copds
                             })

columns = ['gender', 'race', 'age']
categorical = ['gender', 'race', 'age']
groupby = 'COPD'
mytable = TableOne(df, columns=columns, categorical=categorical,
                   groupby=groupby, pval=True)

mytable

In [ ]:
copds_all